In [18]:
import jieba
import pandas as pd

In [67]:
data = pd.read_excel('raw_data/data_gd.xlsx')

In [68]:
data.head()

,来源,来源分类,url,创建时间,情感,作者,标题,内容,评论数,阅读数,点赞数,转发数,是否处理
0,19楼-八卦田,论坛,http://www.19lou.com/forum-125-thread-30114889...,42802.0,正面,NaN,俘获权志龙宋仲基她们是男神最爱女神,俘获权志龙宋仲基她们是男神最爱女神（文/小舞）今天是3月8日女神节我们来盘点一下娱乐圈的女神...,0,0.0,0,NaN,0.0
1,19楼-八卦田,论坛,http://www.19lou.com/forum-125-thread-44641488...,42802.0,正面,NaN,有奇怪收藏癖好的他们权志龙爱名鞋泫雅喜欢它,有奇怪收藏癖好的他们权志龙爱名鞋泫雅喜欢它6日播出的《名单公开2017》中公开了喜欢收集东西...,0,0.0,0,NaN,0.0
2,19楼-八卦田,论坛,http://www.19lou.com/forum-125-thread-65014885...,42797.0,正面,NaN,又揽一员大将！李洙赫加入YG成权志龙同门,又揽一员大将！李洙赫加入YG成权志龙同门3日下午YG官方发表了与演员李洙赫签订专属合约的消息...,0,0.0,0,NaN,0.0
3,19楼-八卦田,论坛,http://www.19lou.com/forum-125-thread-80414870...,42780.0,正面,NaN,宋仲基杨洋张艺兴你想要谁的玫瑰花？,宋仲基杨洋张艺兴你想要谁的玫瑰花？今天是情人节你脱单了吗？没有脱单也不要紧小编找到了中韩男神...,0,0.0,0,NaN,0.0
4,19楼-八卦田,论坛,http://www.19lou.com/forum-125-thread-47114889...,42802.0,负面,NaN,太阳接棒TOP购昂贵名画权志龙损其冲动消费,太阳接棒TOP购昂贵名画权志龙损其冲动消费7日晚太阳通过INS分享了新入手的来自Ryansu...,0,0.0,0,NaN,0.0


In [16]:
len('asfasf')

6

In [23]:
data.shape

(5012, 13)

In [63]:
class emotion_classifier:
    '''
    对一句话的情感进行分类的类
    
    '''
    def __init__(self):
        jieba.load_userdict('resources/emotion_user_dict.txt')
        self.neg_dict = set(self.load_txt('resources/full_neg_dict_sougou.txt'))
        self.pos_dict = set(self.load_txt('resources/full_pos_dict_sougou.txt'))
        inter = self.neg_dict & self.pos_dict # "& | ~" 与 或 非
        self.neg_dict = self.neg_dict - inter
        self.pos_dict = self.pos_dict - inter
        self.deny_set = set(['不', '不是', '否'])
        self.degree_map = self.load_degree_map('resources/degree_dict.xlsx')
    
    def load_degree_map(self, file_path):
        degree_dict = pd.read_excel(file_path)
        degree_map = {x: y for x, y in zip(degree_dict.word, degree_dict.strength)} # zip(..., ..., ...) 对多个series或list同时进行遍历
        return degree_map
    
    def load_txt(self, file_path):
        '''
        加载txt
        with <exp1> as <exp2> == <exp2> = <exp1>
        做完with as 操作后，自动删除加载的文档
        strip() 去掉字符串左右空格

        '''
        result = []
        with open(file_path, 'r') as f:
            for line in f:
                result.append(line.strip())
        return result
    
    def naive_emotion_analyse(self, sentence):
        seg = jieba.lcut(sentence)
        pos_score = len([x for x in seg if x in self.pos_dict])
        neg_score = len([x for x in seg if x in self.neg_dict])
        return pos_score - neg_score
    
    def advance_emotion_analyse(self, sentence):
        seg = jieba.lcut(sentence)
        emotion_list = []
        status = 1
        for word in seg:
            status = status * self.degree_map.get(word, 1)
            if word in self.deny_set:
                status *= -1
            elif word in self.pos_dict:
                emotion_list.append(status)
                status = 1
            elif word in self.neg_dict:
                emotion_list.append(-status)
                status = 1
            else:
                pass
        return sum(emotion_list)
        


In [64]:
worker = emotion_classifier()
sentence = '我今天心旷神怡'
score = worker.naive_emotion_analyse(sentence)
print('语句: {} \n得分: {}'.format(sentence, score))

语句: 我今天心旷神怡 
得分: 1


In [7]:
load_txt('resources/full_neg_dict_sougou.txt')

['脏乱',
 '糟报',
 '早衰',
 '责备',
 '贼眼',
 '战祸',
 '招灾',
 '折辱',
 '中山狼',
 '诪张',
 '株连',
 '坠毁',
 '滋生',
 '自恃',
 '走后门',
 '阻力',
 '罪状',
 '做戏',
 '哀鸿遍野',
 '肮里肮脏',
 '不成话',
 '昭然若揭',
 '有口无心',
 '不干不净',
 '手脚不干净',
 '索然寡味',
 '死板板',
 '松垮垮',
 '危在旦夕',
 '无纪律',
 '无实质',
 '险峭',
 '不明不白',
 '不清不白',
 '不透明',
 '不雅',
 '不足为外人道',
 '玄妙',
 '罪恶多端',
 '皱巴',
 '逞性子',
 '臭名昭彰',
 '哀梨蒸食',
 '度量小',
 '翻云覆雨',
 '腐化堕落',
 '高不成低不就',
 '孤单单',
 '寡断',
 '乖剌',
 '含糊不清',
 '毫不客气',
 '胡子拉碴',
 '荒诞派',
 '荒淫无度',
 '晦暝',
 '昏黑',
 '抱疚',
 '憋着一肚子气',
 '愁惨',
 '掉魂',
 '发牢骚',
 '烦神',
 '忿忿不平',
 '惶然不安',
 '急死',
 '口诛笔伐',
 '骂得狗血喷头',
 '人身攻击',
 '尖嘴薄舌',
 '心如铁石',
 '见风转舵',
 '可骇',
 '枯寂',
 '狂恣',
 '老辣',
 '肋脦',
 '哩哩罗罗',
 '令人厌倦',
 '碌碌无为',
 '马马虎虎',
 '莽莽撞撞',
 '没骨头',
 '没两下子',
 '没心眼儿',
 '模棱两可',
 '少见多怪',
 '难上难',
 '一塌糊涂',
 '只顾自身利益',
 '难驯服',
 '闹闹嚷嚷',
 '铺张浪费',
 '奇丑无比',
 '穷途潦倒',
 '缺乏幽默',
 '弱智',
 '色迷迷',
 '傻里傻气',
 '食古不化',
 '喧天',
 '捉摸不定',
 '滑天下之大稽',
 '上赶',
 '鼠胆',
 '心胸狭隘',
 '性格内向',
 '越轨',
 '哀莫大于心死',
 '猫鼠同处',
 '四海鼎沸',
 '出疆载质',
 '委国听令',
 '国步艰难',
 '溽热',
 '偏疼',
 '娇惯',
 '愤懑',
 '暴跳如雷',
 '戟指怒目',
 '怒形于色',

In [39]:
degree_dict = pd.read_excel('resources/degree_dict.xlsx')
degree_dict.head()

,word,strength
0,百分之百,3.0
1,倍加,3.0
2,备至,3.0
3,不得了,3.0
4,不堪,3.0


In [40]:
degree_map = {x: y for x, y in zip(degree_dict.word, degree_dict.strength)} # zip(..., ..., ...) 对多个series或list同时进行遍历

In [65]:
print(worker.degree_map.get('非常qdsfgbhnmj', 1))
print(worker.degree_map.get('非常', 1))

1
3.0


In [66]:
worker.advance_emotion_analyse('我今天非常开心，我昨天很不开心')

0.5

In [71]:
data['content_all'] = data['标题'].astype(str) + ',' + data['内容'].astype(str)

In [72]:
data.head()

,来源,来源分类,url,创建时间,情感,作者,标题,内容,评论数,阅读数,点赞数,转发数,是否处理,content_all
0,19楼-八卦田,论坛,http://www.19lou.com/forum-125-thread-30114889...,42802.0,正面,NaN,俘获权志龙宋仲基她们是男神最爱女神,俘获权志龙宋仲基她们是男神最爱女神（文/小舞）今天是3月8日女神节我们来盘点一下娱乐圈的女神...,0,0.0,0,NaN,0.0,"俘获权志龙宋仲基她们是男神最爱女神,俘获权志龙宋仲基她们是男神最爱女神（文/小舞）今天是3月..."
1,19楼-八卦田,论坛,http://www.19lou.com/forum-125-thread-44641488...,42802.0,正面,NaN,有奇怪收藏癖好的他们权志龙爱名鞋泫雅喜欢它,有奇怪收藏癖好的他们权志龙爱名鞋泫雅喜欢它6日播出的《名单公开2017》中公开了喜欢收集东西...,0,0.0,0,NaN,0.0,"有奇怪收藏癖好的他们权志龙爱名鞋泫雅喜欢它,有奇怪收藏癖好的他们权志龙爱名鞋泫雅喜欢它6日播..."
2,19楼-八卦田,论坛,http://www.19lou.com/forum-125-thread-65014885...,42797.0,正面,NaN,又揽一员大将！李洙赫加入YG成权志龙同门,又揽一员大将！李洙赫加入YG成权志龙同门3日下午YG官方发表了与演员李洙赫签订专属合约的消息...,0,0.0,0,NaN,0.0,"又揽一员大将！李洙赫加入YG成权志龙同门,又揽一员大将！李洙赫加入YG成权志龙同门3日下午Y..."
3,19楼-八卦田,论坛,http://www.19lou.com/forum-125-thread-80414870...,42780.0,正面,NaN,宋仲基杨洋张艺兴你想要谁的玫瑰花？,宋仲基杨洋张艺兴你想要谁的玫瑰花？今天是情人节你脱单了吗？没有脱单也不要紧小编找到了中韩男神...,0,0.0,0,NaN,0.0,"宋仲基杨洋张艺兴你想要谁的玫瑰花？,宋仲基杨洋张艺兴你想要谁的玫瑰花？今天是情人节你脱单了吗..."
4,19楼-八卦田,论坛,http://www.19lou.com/forum-125-thread-47114889...,42802.0,负面,NaN,太阳接棒TOP购昂贵名画权志龙损其冲动消费,太阳接棒TOP购昂贵名画权志龙损其冲动消费7日晚太阳通过INS分享了新入手的来自Ryansu...,0,0.0,0,NaN,0.0,"太阳接棒TOP购昂贵名画权志龙损其冲动消费,太阳接棒TOP购昂贵名画权志龙损其冲动消费7日晚..."


In [73]:
data['score'] = data['content_all'].map(lambda x: worker.advance_emotion_analyse(x))

In [77]:
data['情感2'] = data['score'].map(lambda x: '正面' if x >= 0 else '负面')

In [78]:
data.head()

,来源,来源分类,url,创建时间,情感,作者,标题,内容,评论数,阅读数,点赞数,转发数,是否处理,content_all,score,情感2
0,19楼-八卦田,论坛,http://www.19lou.com/forum-125-thread-30114889...,42802.0,正面,NaN,俘获权志龙宋仲基她们是男神最爱女神,俘获权志龙宋仲基她们是男神最爱女神（文/小舞）今天是3月8日女神节我们来盘点一下娱乐圈的女神...,0,0.0,0,NaN,0.0,"俘获权志龙宋仲基她们是男神最爱女神,俘获权志龙宋仲基她们是男神最爱女神（文/小舞）今天是3月...",1.0,正面
1,19楼-八卦田,论坛,http://www.19lou.com/forum-125-thread-44641488...,42802.0,正面,NaN,有奇怪收藏癖好的他们权志龙爱名鞋泫雅喜欢它,有奇怪收藏癖好的他们权志龙爱名鞋泫雅喜欢它6日播出的《名单公开2017》中公开了喜欢收集东西...,0,0.0,0,NaN,0.0,"有奇怪收藏癖好的他们权志龙爱名鞋泫雅喜欢它,有奇怪收藏癖好的他们权志龙爱名鞋泫雅喜欢它6日播...",6.0,正面
2,19楼-八卦田,论坛,http://www.19lou.com/forum-125-thread-65014885...,42797.0,正面,NaN,又揽一员大将！李洙赫加入YG成权志龙同门,又揽一员大将！李洙赫加入YG成权志龙同门3日下午YG官方发表了与演员李洙赫签订专属合约的消息...,0,0.0,0,NaN,0.0,"又揽一员大将！李洙赫加入YG成权志龙同门,又揽一员大将！李洙赫加入YG成权志龙同门3日下午Y...",3.0,正面
3,19楼-八卦田,论坛,http://www.19lou.com/forum-125-thread-80414870...,42780.0,正面,NaN,宋仲基杨洋张艺兴你想要谁的玫瑰花？,宋仲基杨洋张艺兴你想要谁的玫瑰花？今天是情人节你脱单了吗？没有脱单也不要紧小编找到了中韩男神...,0,0.0,0,NaN,0.0,"宋仲基杨洋张艺兴你想要谁的玫瑰花？,宋仲基杨洋张艺兴你想要谁的玫瑰花？今天是情人节你脱单了吗...",0.0,正面
4,19楼-八卦田,论坛,http://www.19lou.com/forum-125-thread-47114889...,42802.0,负面,NaN,太阳接棒TOP购昂贵名画权志龙损其冲动消费,太阳接棒TOP购昂贵名画权志龙损其冲动消费7日晚太阳通过INS分享了新入手的来自Ryansu...,0,0.0,0,NaN,0.0,"太阳接棒TOP购昂贵名画权志龙损其冲动消费,太阳接棒TOP购昂贵名画权志龙损其冲动消费7日晚...",2.0,正面


In [79]:
data.sort_values('score', ascending = False)

,来源,来源分类,url,创建时间,情感,作者,标题,内容,评论数,阅读数,点赞数,转发数,是否处理,content_all,score,情感2
3503,微博,微博,http://weibo.com/2612500844/EztHmvbG7,42806.805625,NaN,八方美熊李胖达,NaN,NA#权志龙##全世界最好的权志龙#他真的非常非常好，非常非常善良，非常非常热爱和平?&[心...,0,NaN,0,0.0,NaN,"nan,NA#权志龙##全世界最好的权志龙#他真的非常非常好，非常非常善良，非常非常热爱和平...",92.000,正面
3944,微博,微博,http://weibo.com/1453372141/Ezj5Iq8Ej,42805.680486,NaN,赵盛烨,NaN,NA权志龙之前来中国，杜海涛见到下跪，国家面前无偶像！不用韩妆不会死，不看韩剧不会死，不吃泡...,33,NaN,35,2.0,NaN,"nan,NA权志龙之前来中国，杜海涛见到下跪，国家面前无偶像！不用韩妆不会死，不看韩剧不会死...",81.000,正面
2406,微博,微博,http://weibo.com/6174170527/Ezy5Nt5lX,42807.271736,NaN,我就要直接说,NaN,NA#爱国便要取关明星吗？#现在网上很多人都在喊着我要取关某某某，韩国明星。国家面前无明星之...,10,NaN,14,0.0,NaN,"nan,NA#爱国便要取关明星吗？#现在网上很多人都在喊着我要取关某某某，韩国明星。国家面前...",28.125,正面
3318,微博,微博,http://weibo.com/5948442747/Ezpt9pNr8,42806.357014,NaN,音乐疯子酷爱音乐,NaN,NA这俩人是我最爱的两个人爱豆里的最爱，演员里的最爱#权志龙##李钟硕#是因为他俩人性格都是...,18,NaN,18,0.0,NaN,"nan,NA这俩人是我最爱的两个人爱豆里的最爱，演员里的最爱#权志龙##李钟硕#是因为他俩人...",24.000,正面
4008,微博,微博,http://weibo.com/1968476865/E0zksSzJe,42805.826169,NaN,在你来之前龙卷风席卷了我的房间,NaN,NA权志龙这个人，已经站在了一定的高度，却从不吹嘘自己的成就，做了好事不会得意洋洋广而告之，...,1,NaN,11,2.0,NaN,"nan,NA权志龙这个人，已经站在了一定的高度，却从不吹嘘自己的成就，做了好事不会得意洋洋广...",17.250,正面
2408,微博,微博,http://weibo.com/3595290784/EzcFCjVFV,42804.999387,NaN,龍與勝鉉,NaN,NA#GTOP#嗯~抱着开心…因为换了手机这组照片想找很久了真是特别特别喜欢这场天阿我太想你...,0,NaN,2,0.0,NaN,"nan,NA#GTOP#嗯~抱着开心…因为换了手机这组照片想找很久了真是特别特别喜欢这场天阿...",16.625,正面
2849,微博,微博,http://weibo.com/5948442747/EzhDLmsvB,42805.526574,NaN,音乐疯子酷爱音乐,NaN,NA我龙的潮范真心无人能敌啊，原谅我爱他歌唱的那么好，原谅我爱他舞蹈跳的那么好，原谅我爱他颜...,0,NaN,5,0.0,NaN,"nan,NA我龙的潮范真心无人能敌啊，原谅我爱他歌唱的那么好，原谅我爱他舞蹈跳的那么好，原谅...",16.500,正面
1114,微博,微博,http://weibo.com/5807599912/Ez5Yuj67w,42804.289155,NaN,Perfect男银,NaN,NA#抵制韩国#看一下我们中国在韩国人心里的形象吧！不要再去舔你们的韩国棒子偶像，人傻钱多就...,0,NaN,0,0.0,NaN,"nan,NA#抵制韩国#看一下我们中国在韩国人心里的形象吧！不要再去舔你们的韩国棒子偶像，人...",16.200,正面
5003,微博,微博,http://weibo.com/3164340711/Ez7d9jVyt,42804.420312,NaN,权志龙你干嘛,NaN,NA可爱死了//@馨随龙动:#权志龙#这个脑洞太可爱[笑cry]//@O_GDragon:哈...,0,NaN,0,0.0,NaN,"nan,NA可爱死了//@馨随龙动:#权志龙#这个脑洞太可爱[笑cry]//@O_GDrag...",16.000,正面
1557,微博,微博,http://weibo.com/2721270145/Ez8iPEbYZ,42804.536111,NaN,M_ONSANT_,NaN,NA太帅气了[爱你]//@年度最佳龙骑士:我们基地哥哥太好看了吧//@龙宝的迷妹:我最爱最爱...,0,NaN,1,0.0,NaN,"nan,NA太帅气了[爱你]//@年度最佳龙骑士:我们基地哥哥太好看了吧//@龙宝的迷妹:我...",16.000,正面
